In [ ]:
import pandas as pd
import requests
import os

In [ ]:
nasdaq_tickers = pd.read_csv('../data/raw/nasdaq_tickers.txt', sep='|')
nyse_tickers = pd.read_csv('../data/raw/other_tickers.txt', sep='|')
nyse_tickers.Exchange.unique()
API_KEY = os.getenv("API_KEY")

array(['N', 'P', 'Z', 'A', 'V', nan], dtype=object)

In [ ]:
# exchange_map = {
#     'A': 'AMEX',
#     'N': 'NYSE',
#     'P': 'ARCA',
#     'V': 'IEX',
#     'Z': 'BATS'
# }

# nyse_tickers['Exchange'] = nyse_tickers['Exchange'].map(exchange_map)
nyse_tickers.Exchange.unique()

In [25]:
# Cleaning other tickers csv
nyse_tickers = nyse_tickers[nyse_tickers['Test Issue'] == 'N']
nyse_tickers = nyse_tickers[nyse_tickers['ETF'] == 'N']
# change exchange letters to full exchange names for reability
exchange_map = {
    'A': 'AMEX',
    'N': 'NYSE',
    'P': 'ARCA',
    'V': 'IEX',
    'Z': 'BATS'
}

nyse_tickers['Exchange'] = nyse_tickers['Exchange'].map(exchange_map)
nyse_tickers

,ACT Symbol,Security Name,Exchange,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
0,A,"Agilent Technologies, Inc. Common Stock",NYSE,A,N,100.0,N,A
1,AA,Alcoa Corporation Common Stock,NYSE,AA,N,100.0,N,AA
4,AACT,Ares Acquisition Corporation II Class A Ordina...,NYSE,AACT,N,100.0,N,AACT
5,AACT.U,"Ares Acquisition Corporation II Units, each co...",NYSE,AACT.U,N,100.0,N,AACT=
6,AACT.W,Ares Acquisition Corporation II Redeemable War...,NYSE,AACT.WS,N,100.0,N,AACT+
...,...,...,...,...,...,...,...,...
6484,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,NYSE,ZTO,N,100.0,N,ZTO
6485,ZTR,Virtus Total Return Fund Inc.,NYSE,ZTR,N,100.0,N,ZTR
6486,ZTS,Zoetis Inc. Class A Common Stock,NYSE,ZTS,N,100.0,N,ZTS
6487,ZVIA,Zevia PBC Class A Common Stock,NYSE,ZVIA,N,100.0,N,ZVIA


In [26]:
others_columns_to_drop = ['CQS Symbol', 'ETF', 'Round Lot Size', 'Test Issue']
nyse_tickers.drop(labels=others_columns_to_drop, axis=1, inplace=True)
nyse_tickers

,ACT Symbol,Security Name,Exchange,NASDAQ Symbol
0,A,"Agilent Technologies, Inc. Common Stock",NYSE,A
1,AA,Alcoa Corporation Common Stock,NYSE,AA
4,AACT,Ares Acquisition Corporation II Class A Ordina...,NYSE,AACT
5,AACT.U,"Ares Acquisition Corporation II Units, each co...",NYSE,AACT=
6,AACT.W,Ares Acquisition Corporation II Redeemable War...,NYSE,AACT+
...,...,...,...,...
6484,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,NYSE,ZTO
6485,ZTR,Virtus Total Return Fund Inc.,NYSE,ZTR
6486,ZTS,Zoetis Inc. Class A Common Stock,NYSE,ZTS
6487,ZVIA,Zevia PBC Class A Common Stock,NYSE,ZVIA


In [ ]:

nyse_tickers.set_index('ACT Symbol')

In [ ]:
# Clean nasdaq ticker data
nasdaq_tickers = nasdaq_tickers[nasdaq_tickers['Test Issue'] == 'N']
nasdaq_tickers = nasdaq_tickers[nasdaq_tickers['ETF'] == 'N']
nasdaq_tickers = nasdaq_tickers[nasdaq_tickers['NextShares'] == 'N']
nasdaq_tickers['Exchange'] = 'NASDAQ'
# nasdaq_tickers
# Still 1 missing ticker symbol
# nasdaq_tickers[nasdaq_tickers['Symbol'].isnull()]
# after googling, found the symbol is NA for this company, no other stock has that ticker so will replace the null with NA row id 2894
nasdaq_tickers['Symbol'] = nasdaq_tickers['Symbol'].fillna('NA')
nasdaq_tickers

In [ ]:
nasdaq_columns_to_drop = ['Market Category', 'Test Issue', 'Financial Status', 'Round Lot Size', 'ETF', 'NextShares']
nasdaq_tickers.drop(labels=nasdaq_columns_to_drop, axis=1, inplace=True)
nasdaq_tickers

In [ ]:
nyse_tickers

In [ ]:
# Join 

# Ensure both have 'Ticker' as a column
nasdaq_tickers['Ticker'] = nasdaq_tickers['Symbol']
nyse_tickers['Ticker'] = nyse_tickers['ACT Symbol']

# Check for overlap
duplicate_tickers = set(nasdaq_tickers['Ticker']) & set(nyse_tickers['Ticker'])
print(f"Number of duplicate tickers: {len(duplicate_tickers)}")
print("Some duplicate tickers:", list(duplicate_tickers)[:10])

In [ ]:
# Trying out fundamental data requests

sample_tickers_list = ['AAPL', 'AACB', 'AAPD', 'ABLLL', 'HPKEW', 'HPK', 'NMRA', 'PLSE', 'PNTG', 'SEEM']
json_list = []
for ticker in sample_tickers_list:
    url = f'https://financialmodelingprep.com/api/v3/enterprise-values/{ticker}/?period=quarter&apikey={API_KEY}&limit=1'
    res = requests.get(url)
    data = res.json()
    json_list.append(data)
pd.DataFrame(json_list)

,0
0,"{'symbol': 'AAPL', 'date': '2025-03-29', 'stoc..."
1,None
2,None
3,"{'symbol': 'ABLLL', 'date': '2024-12-31', 'sto..."
4,"{'symbol': 'HPKEW', 'date': '2024-12-31', 'sto..."
5,"{'symbol': 'HPK', 'date': '2024-12-31', 'stock..."
6,"{'symbol': 'NMRA', 'date': '2024-12-31', 'stoc..."
7,"{'symbol': 'PLSE', 'date': '2024-12-31', 'stoc..."
8,"{'symbol': 'PNTG', 'date': '2024-12-31', 'stoc..."
9,None


In [ ]:
# Trying out fundamental data requests

url = f'https://financialmodelingprep.com/api/v3/enterprise-values?limit=10&apikey={API_KEY}'
res = requests.get(url)
data = res.json()
pd.DataFrame(data)

""


In [21]:
# Gets tickers and basic data from 85258 companies
url = 'https://financialmodelingprep.com/api/v3/stock/list?apikey=b95993f82e2d0048935cfd947df28088'
res = requests.get(url)
data = res.json()
df = pd.DataFrame(data)
df = df[df['price'] >= 2.5]
df = df[df['type'] == 'stock']
df

,symbol,name,price,exchange,exchangeShortName,type
0,PSMT,"PriceSmart, Inc.",103.800,NASDAQ Global Select,NASDAQ,stock
1,ADPT,Adaptive Biotechnologies Corporation,9.295,NASDAQ Global Select,NASDAQ,stock
2,SEDG,"SolarEdge Technologies, Inc.",19.460,NASDAQ Global Select,NASDAQ,stock
3,NKE,"NIKE, Inc.",58.630,New York Stock Exchange,NYSE,stock
4,AROC,"Archrock, Inc.",24.400,New York Stock Exchange,NYSE,stock
...,...,...,...,...,...,...
85242,NALA.TA,Nala Digital Commerce Ltd,43.300,Tel Aviv,TLV,stock
85243,SDACU,Sustainable Development Acquisition I Corp.,10.375,NASDAQ Capital Market,NASDAQ,stock
85244,3308.HK,Golden Eagle Retail Group Limited,6.840,Hong Kong Stock Exchange,HKSE,stock
85249,UNVR,Univar Solutions Inc.,36.140,New York Stock Exchange,NYSE,stock
